<a href="https://colab.research.google.com/github/nupursjsu/AlternusVera/blob/master/FourReal_AV_FinalCombinedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Alternus Vera - Final Combined Model**

Doing multiclass classification between 1 and 6, 1 being true and 6 being pants-fire.

We have also performed LDA as part of distillation on liar liar, kaggle fake news and kaggle fake news detection datasets. But to keep this notebook concise we haven't included that here. Link given below.

[LDA on datasets](https://drive.google.com/open?id=1eI7Ty9AMRL1Wii4lb7us16gYc88MTkD4)

#**Team Name : FourReal**

GitHub URL: https://github.com/nupursjsu/AlternusVera

### **Team Contributions:**

|Features  |  Member |
|-----|-----|
| Political Affiliation                         |  Nupur Yadav |  
| Title Vs Body                   |  Lokesh Vadlamudi  |  
|   Network Factor                           |  Ronak Mehta 
| Stance Model                 |  Chetan Kulkarni |  


### **Enrichment Dataset Details:**

- Kaggle Fakenews Dataset
- Kaggle News Detection
- Politifact Fake news and Real News Content
- SemEval Dataset
- Fake News Challenge


### **Libraries Used:**

- NLTK 
- Gensim 
- Fastai
- Numpy
- Pandas
- CSV
- WordCloud
- Seaborn
- Scipy
- Regualr Expression
- Matplotlib
- Sklearn 

**To keep the notebook short and concise we have just imported our already trained models for our individual factors. The links to each individual factors colabs can be found below.**

[Political Affiliation](https://drive.google.com/open?id=1UMq9LCX3A20JNMW_bxQl_HRMiTARNrOm)

[Network Based](https://drive.google.com/open?id=1nkBr35YVMkWb8JLZeJivl_xQBIXZMWlX)

[Title Vs Body](https://drive.google.com/open?id=1g5G8UzV6iZY9wLLechpWpNaE5PzldozH)

[Stance Factor](https://drive.google.com/open?id=1PVqhXDSrMVOq6BswfpBz7c6PZCoYS_7x)


#**Importing necessary libraries**


In [29]:
import warnings
warnings.filterwarnings("ignore")

!pip install nltk
!pip install gensim
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import numpy as np
import pandas as pd
import nltk
import gensim
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pickle
from fastai import *
from fastai.tabular import *



import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
# from nltk.corpus import stopwords
# nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

# NLTK libraries
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
import re

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import joblib

print(tf.__version__)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2.2.0


#**Feature 1 : Political Affiliation Factor**

In [0]:
# Read data file into colaboratory
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#Function to load already trained models and make prediction
def FourReal_getPartyAffiliationScore(statement, speaker_job_title, state_info, context): # return between 1 and 6, being 1 = true,  6 = pants-fire
  if ( (statement == "") | (speaker_job_title == "") | (state_info == "") | (context == "")):
    return 0

  #loading the doc2vec model from drive to generate embeddings for statement
  link = 'https://drive.google.com/open?id=1G0GC1VT5ogKyUcMSbdpivDVtPV6Lhheq'
  fluff, id = link.split('=')
  
  #downloading the doc2vec model
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('doc2vec_model.sav')
  d2vmodel = pickle.load(open('doc2vec_model.sav', 'rb'))
  
  statement_wordvec = [d2vmodel.infer_vector(word_tokenize(statement))]
  
  #creating the dataframe with our statement embeddings and other categorical columns 
  #to get a party affiliation probability vector for our final prediction
  df = pd.DataFrame(statement_wordvec)
  df['speaker_job_title']= speaker_job_title
  df['state_info'] = state_info
  df['context'] = context
  df.columns = ['Embed_0', 'Embed_1', 'Embed_2', 'Embed_3', 'Embed_4', 'Embed_5', 'Embed_6', 'Embed_7','Embed_8', 'Embed_9', 'speaker_job_title', 'state_info', 'context']
      
  #Loading our NN model to get party affiliation probability vector
  link = 'https://drive.google.com/open?id=1i3TknJOHp-kxUuqL9S2nX1jb-3EQVo5F'
  fluff, id = link.split('=')
  
  #downloading the NN model
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('export.pkl')
  NN_model = load_learner('/content')

  pred = NN_model.predict(df.iloc[0])
  pf_pred = pred[0] 
  pf_probability_vector = pred[2] #party affiliation probability vector

  #creating dataframe for our final prediction between 1-6
  #converting vector to dataframe columns
  v = []
  for i in pf_probability_vector:
    a = float(i)
    v.append(a)

  columns = ['Prob_republican','Prob_democrat', 'Prob_other']
  final_df = pd.DataFrame(v).transpose()
  final_df.columns = columns

  #Loading LR model to make final prediction where a statement lies between 1-6 with 1 being true and 6 being pants-fire
  link = 'https://drive.google.com/open?id=1CaBMxljH5Z7FVXude4btyb5KWGoHNDZ2'
  fluff, id = link.split('=')
  
  #downloading the doc2vec model
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile('LogisticRegression.pkl')
  pa_model = joblib.load('LogisticRegression.pkl')

  predicted = pa_model.predict(final_df)
  # predicedProb = pa_model.predict_proba(final_df)[:,1]
  predicedProb=pa_model.predict_proba(final_df)
  return predicedProb.tolist()[0]     #returns probability vector from 1 to 6, with 1 being true and 6 being pants-fire


In [32]:
print(FourReal_getPartyAffiliationScore("Says the Annies List political group supports third-trimester abortions on demand", "State representative",	"Texas", "a mailer"))

[0.18341061950598983, 0.23215116583821807, 0.19679668628356797, 0.16299004866876748, 0.08077991966860482, 0.14387156003485185]


#**Feature 2 : Title Vs Body**

In [0]:
link = 'https://drive.google.com/open?id=1r2LPuNUKy_qMlLzfDnIVdeVZfZrpOngF'
fluff, id = link.split('=')

#downloading the doc2vec model
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('lokiXgb.pkl')
xgbLoki = joblib.load(open('lokiXgb.pkl', 'rb'))

In [0]:
link = 'https://drive.google.com/open?id=1qoqFk7BU_OCMroq_vvcdWDGzm8w-VDmk'
fluff, id = link.split('=')

#downloading the doc2vec model
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('party.pkl')
party = joblib.load(open('party.pkl', 'rb'))

In [0]:
def predict_tVsBScore(title,body):
    # import string
    # from nltk.stem import WordNetLemmatizer
    # from nltk.tokenize import RegexpTokenizer
    # from nltk.corpus import stopwords
    # nltk.download('stopwords')
    # from nltk.stem.porter import PorterStemmer


    stemming = PorterStemmer()
    lemma = WordNetLemmatizer()
    def cleanUp(data):
        data = "".join([i for i in str(data) if i not in string.punctuation])
        tokenizer = RegexpTokenizer(r'\w+')
        data = tokenizer.tokenize(data.lower())
        data = [i for i in data if i not in stopwords.words('english')]
        data = [lemma.lemmatize(i) for i in data]
        data = " ".join([stemming.stem(i) for i in data])
        # print(data)
        return data

    # import re
    def break_subject(text):
        if type(text) is float:
          return []
        import re
        a = re.split('; |, |-',text)
        # print(a)
        return re.split('; |, |-',text)

    cleanedBody = cleanUp(body)
    subject = break_subject(title)

    from nltk.corpus import wordnet 
    def generate_Synonyms(wordlist):
        synonyms = set()
        for i in wordlist:
          if type(i) is not str:
            continue
          for syn in wordnet.synsets(i): 
              for l in syn.lemmas(): 
                  synonyms.add(l.name())
        # print(list(synonyms))
        return list(synonyms)
    def combineSynonyms(data):
        return ' '.join(data)
    

    finalSubject = generate_Synonyms(subject)
    # print(finalSubject)
    finalSubject1 = combineSynonyms(finalSubject)
    # print(finalSubject1)

    s = cleanedBody + '@@' + finalSubject1
    # print(s)

    from sklearn.feature_extraction.text import TfidfVectorizer
    # create the transform
    def cosine_sim(statement):
        statement=statement.split('@@')
        # print(statement)
        text1,text2=statement[0],statement[1]
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf = vectorizer.fit_transform([text1, text2])
        sim_score=((tfidf * tfidf.T).A)[0,1]
        pred_label = 0
        if sim_score*100>20:
          pred_label = 1
          
        # if sim_score*100>0:
        #   return 1
        return sim_score , pred_label

    import spacy
    
    # Load English tokenizer, tagger, parser, NER and word vectors
    nlp = spacy.load('en')
    def spacy_similarity(t1,t2):
        doc1 = nlp(t1)
        doc2 = nlp(t2)
        return doc1.similarity(doc2)

    
    simScore , label = cosine_sim(s)
    simScore2 = spacy_similarity(cleanedBody,finalSubject1)
    
    return label,simScore2

In [0]:
def FourReal_TitleVsBody(title,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts, pants_on_fire_counts,party_affiliation):
  # cosineSim = predict_tVsBScore(title,statement)
  party_affiliation_transform = party.transform([party_affiliation])
  
  df=pd.DataFrame([barely_true_counts])
  df.columns=['barely_true_counts']
  df['false_counts']=false_counts
  df['half_true_counts']=half_true_counts
  df['mostly_true_counts']=mostly_true_counts
  df['pants_on_fire_counts']=pants_on_fire_counts
  df['party affiliation']=party_affiliation_transform
  a,df['cosineSim']=predict_tVsBScore(title,statement)
  
  # barely_true_counts	false_counts	half_true_counts	mostly_true_counts	pants_on_fire_counts	party affiliation	cosineSim	label
  
  prediction = xgbLoki.predict(df)
  prediction_probability = xgbLoki.predict_proba(df)
  return prediction_probability.tolist()[0]

#**Feature 3 : Network Factor**

In [0]:
# Google drive link where the dataset is stored.
link = 'https://drive.google.com/open?id=1OckRQ_VK1Ljrabnn9J9x7q3x95Du2Vb_'
fluff, id = link.split('=')
# Download the tsv file.
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('model.hdf5')

In [0]:
def FourReal_getNetwordBasedScore(text):
  text = re.sub("[^a-zA-Z]", " ", str(text))
  text =  text.lower()
  text_words = nltk.word_tokenize(text)
  stops = set(nltk.corpus.stopwords.words("english"))
  words = [w for w in  text_words  if not w in stops]
  wordnet_lem = [WordNetLemmatizer().lemmatize(w) for w in words ]
  stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
  text = " ".join(stems)
  model = tf.keras.models.load_model('model.hdf5')
  tokenizer = Tokenizer(num_words=300, lower=True)
  tokenizer.fit_on_texts(text)
  test_sequences = tokenizer.texts_to_sequences(text)
  test_padded_sequences = sequence.pad_sequences(test_sequences, maxlen=300)
  result = model.predict_classes(test_padded_sequences)
  true,fake = 0,0
  total_occurrences = np.size(result)
  for i in result:
    if i==0:
      true+=1
    else:
      fake+=1
  value = fake/total_occurrences
  if value<=0.167:
    return [0,0,0,0,0,1]
  elif value>0.167 and value<=0.334:
    return [0,0,0,0,1,0]
  elif value>0.334 and value<=0.501:
    return [0,0,0,1,0,0]
  elif value>0.501 and value<=0.668:
    return [0,0,1,0,0,0]
  elif value>0.668 and value<=0.835:
    return [0,1,0,0,0,0]
  else:
    return [1,0,0,0,0,0]
  # print("Probability of text being true:",(true/total_occurrences),"Probability of text being fake:",(fake/total_occurrences))

In [39]:
FourReal_getNetwordBasedScore("Says the Annies List political group supports third-trimester abortions on demand.")

[0, 0, 1, 0, 0, 0]

#**Feature 4 : Stance Model**

In [40]:
from sklearn.externals import joblib 
import re

!pip install nltk
import nltk


nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords


from nltk.tokenize import word_tokenize
import pandas as pd

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
link = 'https://drive.google.com/open?id=1Gct0uLHt2x6jVt1SmcqlyyLPGhZu_Hfe'
fluff, id = link.split('=')

#downloading the doc2vec model
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('le_speaker_job_title_code.pkl')

In [0]:
link = 'https://drive.google.com/open?id=1pOEibOMRmNSHUZIHm0wdJfp3SgHRQ4Q2'
fluff, id = link.split('=')

#downloading the doc2vec model
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('le_speaker.pkl')

In [0]:
link = 'https://drive.google.com/open?id=1EqvpovqAxnKlu50oQCj_ryVF5fOxtilW'
fluff, id = link.split('=')

#downloading the doc2vec model
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('RForest.pkl')

In [0]:
link = 'https://drive.google.com/open?id=1Wb8JEYXqYcifnTsMgamv8HaCzP_Vvxp2'
fluff, id = link.split('=')

#downloading the doc2vec model
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('le_state_info_code.pkl')

In [0]:
link = 'https://drive.google.com/open?id=1-SjP9druNndPShP9fSmtibvWel0FGgqP'
fluff, id = link.split('=')

#downloading the doc2vec model
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('vectorizer.pkl')

In [0]:
link = 'https://drive.google.com/open?id=1iY7WghgNCFq8EYStlYNlqqFuRADpjsGa'
fluff, id = link.split('=')

#downloading the doc2vec model
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('XGB.pkl')

In [0]:
vector=joblib.load('vectorizer.pkl')
loaded_model=joblib.load('XGB.pkl')

le_speaker=joblib.load('le_speaker.pkl')
le_speaker_job_title_code=joblib.load('le_speaker_job_title_code.pkl')
le_state_info_code=joblib.load('le_state_info_code.pkl')
model=joblib.load('XGB.pkl')

In [0]:
stop_words = set(stopwords.words('english')) 
vector=joblib.load('vectorizer.pkl')
loaded_model=joblib.load('RForest.pkl')
def get_stance(statement):
  p_statement= re.sub(r"\W", " ",str(statement))
  p_statement=re.sub(r'[,\.!?]', " ",str(p_statement))
  p_statement=re.sub(r" \d+", " ",str(p_statement))
  p_statement=word_tokenize(p_statement.lower())
  p_statement=' '.join(p_statement)
  p_statement=' '.join([word for word in p_statement.split() if word not in stop_words])
  liar_x = vector.transform([p_statement,])
  stance=loaded_model.predict(liar_x)
  return stance[0]

In [0]:
def FourReal_getStanceScore(statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts, pants_on_fire_counts,speaker,state_info,speaker_job_title):
  speaker_job_title=le_speaker_job_title_code.transform([speaker_job_title])
  speaker=le_speaker.transform([speaker])
  state_info=le_state_info_code.transform([state_info])
  df=pd.DataFrame([barely_true_counts])
  df.columns=['barely_true_counts']
  df['false_counts']=false_counts
  df['half_true_counts']=half_true_counts
  df['mostly_true_counts']=mostly_true_counts

  df['pants_on_fire_counts']=pants_on_fire_counts
  df['Stance']=get_stance(statement)
  df['speaker_code']=speaker
  df['state info_code']=state_info
  df['speaker_job_title_code']=speaker_job_title
 
  # X=np.array(barely_true_counts,false_counts,half_true_counts,mostly_true_counts, pants_on_fire_counts,Stance,speaker,speaker_job_title,state_info)
  # ['barely_true_counts', 'false_counts', 'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts', 'Stance', 'speaker_code', 'state info_code', 'speaker_job_title_code'] 
  # ['barely_true_counts', 'pants_on_fire_counts', 'speaker_code', 'mostly_true_counts', 'Stance', 'half_true_counts', 'false_counts', 'state info_code', 'speaker_job_title_code']
  prediction=model.predict(df)
  prediction_probability=model.predict_proba(df)
  return prediction_probability.tolist()[0]

In [50]:
print(FourReal_getStanceScore('i am awsome',10,10,10,10, 10,'dwayne-bohac','Virginia','State representative'))

[0.1963309645652771, 0.15024597942829132, 0.17514757812023163, 0.17656482756137848, 0.14511354267597198, 0.1565970778465271]


#**Final Combined Model**

**Since, we are doing a multiclass classification, each of our individual models are generating six probabilities scores for 1 to 6 (1 being true and 6 being pants-fire) class labels and then we are doing a weighted average for every probability and at last returning the max probability with the corresponding index denoting a label from 1-6 (here indices from 0 to 5).**

In [0]:
def isFakeNews(statement,subject,speaker_job_title,state_info,context,barely_true_counts,false_counts,half_true_counts,mostly_true_counts, pants_on_fire_counts,speaker,party_affiliation):
  accur = [0.22,0.43,0.43,0.95] # using the (normalized) accuracy as weigths
  w = [float(i)/sum(accur) for i in accur]
  sumW = 0
  prob = []
  if ( (statement != "") & (speaker_job_title != "") & (state_info != "") & (context != "")):
    a1,b1,c1,d1,e1,f1 =  FourReal_getPartyAffiliationScore(statement, speaker_job_title, state_info, context)

  if ((statement != "") & (subject != "")):
    a2,b2,c2,d2,e2,f2 =  FourReal_TitleVsBody(subject,statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts, pants_on_fire_counts,party_affiliation)
    
  if (statement!='' and barely_true_counts!='' and false_counts!='' and half_true_counts!='' and mostly_true_counts!='' and  pants_on_fire_counts!='' and speaker!='' and state_info!='' and speaker_job_title!=''):
    a3,b3,c3,d3,e3,f3 =  FourReal_getStanceScore(statement,barely_true_counts,false_counts,half_true_counts,mostly_true_counts, pants_on_fire_counts,speaker,state_info,speaker_job_title)
    
  if (statement != ""):
    a4,b4,c4,d4,e4,f4 =  FourReal_getNetwordBasedScore(statement)


  A1=a1*w[0]+a2*w[1]+a3*w[2]+a4*w[3] /sum(accur)
  A2=b1*w[0]+b2*w[1]+b3*w[2]+b4*w[3] /sum(accur)
  A3=c1*w[0]+c2*w[1]+c3*w[2]+c4*w[3] /sum(accur)
  A4=d1*w[0]+d2*w[1]+d3*w[2]+d4*w[3] /sum(accur)
  A5=e1*w[0]+e2*w[1]+e3*w[2]+e4*w[3] /sum(accur)
  A6=f1*w[0]+f2*w[1]+f3*w[2]+f4*w[3] /sum(accur)

  prob_list = [A1,A2,A3,A4,A5,A6]
  prob_max = max(prob_list)
  label = prob_list.index(prob_max)

  return label


In [52]:
result = isFakeNews("Hillary Clinton agrees with John McCain by voting to give George Bush the benefit of the doubt on Iran","foreign-policy","President","Illinois","Denver",70,71,160,163,9,'barack-obama','democrat')
print(result)

2


The result is '2' which means 'half-true'